In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import requests 
import time 
df=pd.read_csv("Levels_Fyi_Salary_Data.csv", delimiter=",")
df.info()
df.head(60)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                62642 non-null  object 
 1   company                  62637 non-null  object 
 2   level                    62523 non-null  object 
 3   title                    62642 non-null  object 
 4   totalyearlycompensation  62642 non-null  int64  
 5   location                 62642 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   tag                      61788 non-null  object 
 9   basesalary               62642 non-null  int64  
 10  stockgrantvalue          62642 non-null  float64
 11  bonus                    62642 non-null  float64
 12  gender                   43102 non-null  object 
 13  otherdetails             40137 non-null  object 
 14  cityid                

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.50,NaN,107000,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.00,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.00,NaN,155000,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.00,NaN,157000,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.00,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
5,6/21/2017 17:27,Microsoft,63,Software Engineer,208000,"Seattle, WA",8.5,8.50,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
6,6/22/2017 12:37,Microsoft,65,Software Engineering Manager,300000,"Redmond, WA",15.0,11.00,NaN,180000,...,0,0,0,0,0,0,0,0,NaN,NaN
7,6/22/2017 13:55,Microsoft,62,Software Engineer,156000,"Seattle, WA",4.0,4.00,NaN,135000,...,0,0,0,0,0,0,0,0,NaN,NaN
8,6/22/2017 23:08,Microsoft,59,Software Engineer,120000,"Redmond, WA",3.0,1.00,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
9,6/26/2017 21:25,Microsoft,63,Software Engineer,201000,"Seattle, WA",12.0,6.00,NaN,157000,...,0,0,0,0,0,0,0,0,NaN,NaN


In [2]:
#Connor
# take the location column and seperate state and city and index them, take min/max of base salary/totalyearlycompensation 

#removing salaries of "0"
df_cleaned = df[df['basesalary'] != 0]
df_cleaned.head(20)
df_cleaned['basesalary'].min()
df_cleaned['basesalary'].max()
df_cleaned['totalyearlycompensation'].min()
df_cleaned['totalyearlycompensation'].max()

#created a cleaner DF that created invidual lists of each company and their average salary
#note: some companies are still repeated more than once (IE: "7-Eleven" & "7-eleven",etc)

#company_data = "../Levels_Fyi_Salary_Data.csv"

company_name = df_cleaned.set_index('company').groupby(['company'])
                                
average_salary = company_name['basesalary'].mean()

average_total_salary = company_name['totalyearlycompensation'].mean()
                                       
company_summary_df = pd.DataFrame({
    "Company Name": company_name,
    "Average Total Salary": average_total_salary,
    "Average Salary": average_salary,
   
})

company_summary_df = company_summary_df[['Average Salary', 'Average Total Salary']]


#cleaning the DF
company_summary_df.style.format({
    "Average Salary": "${:,.2f}",
    "Average Total Salary": "${:,.2f}",
})

company_summary_df['Average Salary'].min()
print (f"The lowest average salary is )
company_summary_df['Average Salary'].max()

,Average Salary,Average Total Salary
company,,
10x Genomics,"$149,666.67","$193,333.33"
23andMe,"$135,833.33","$155,666.67"
2U,"$129,142.86","$145,571.43"
3M,"$107,190.48","$118,857.14"
3m,"$108,666.67","$115,666.67"
7-Eleven,"$117,500.00","$126,750.00"
7-eleven,"$78,000.00","$80,000.00"
8x8,"$176,428.57","$296,571.43"
ABB,"$84,428.57","$94,428.57"


In [ ]:
#take base salary/totalyearlycompensation 

In [ ]:
#group by company and min/ max yearly compensation

In [ ]:
#Education compared to salary/totalyearlycompensation 

In [ ]:
#Alfredo
#years of experience/ years at company and comparison to yearly compensation
df_exp = pd.DataFrame(df,columns=["yearsofexperience","yearsatcompany","basesalary","stockgrantvalue","bonus","totalyearlycompensation"])
print(df_exp["totalyearlycompensation"].min())
print(df_exp["totalyearlycompensation"].max())
print(df_exp["yearsatcompany"].min())
print(df_exp["yearsatcompany"].max())
yearlycomp=df_exp["totalyearlycompensation"]
industryyears=df_exp["yearsofexperience"]
companyyears=df_exp["yearsatcompany"]
df_exp.head()
plt.scatter(companyyears,yearlycomp,marker="o", facecolors="red", edgecolors="black")
plt.scatter(industryyears,yearlycomp,marker="o", facecolors="red", edgecolors="black")
#Find and remove the outliers of both years and pay (Maybe top 100?)
#Check if other columns are usable and, if so, remove the zeroes from them

In [ ]:
plt.scatter(industryyears,yearlycomp,marker="o", facecolors="red", edgecolors="black")

In [ ]:
#Neel
#male/ female convert to usable data and compare to yearly compensation
df.columns
df['gender'].unique()

In [ ]:
# Filtered DataFrame to only include Gender , Job Title, Base Salary, & Yearly Compensation. (4 Columns)
gender_df = df[["gender", "title", "basesalary", "totalyearlycompensation"]]
#gender_df

# Remove NaN values from Gender and remove base salaries that are 0
gender_df = gender_df.dropna()
filt_gender_df = gender_df.loc[gender_df["basesalary"] > 0, ["gender", "title", "basesalary", "totalyearlycompensation"]]
#filt_gender_df.sort_values(by=['gender'])

# Remove 'Title: Senior Software Engineer'
filt_gender_df = filt_gender_df.drop(index=11010)
filt_gender_df.reset_index(drop  = True)

In [ ]:
# Pie Chart of Genders in STEMS
gender = ['Male', 'Female', 'Other']
count = filt_gender_df['gender'].value_counts()
colors = ['coral', 'plum', 'silver']
explode = (0, 0.05, 0.1)

plt.title("STEM Genders")
plt.pie(count, explode=explode, labels = gender, colors = colors, autopct = "%1.1f%%", shadow = True)
plt.axis("equal")
plt.show
plt.savefig("output_images/Genders_In_Stem.png")
count